In [ ]:
import qutip as qt
import numpy as np

Loosly based on https://nbviewer.org/urls/qutip.org/qutip-tutorials/tutorials-v5/time-evolution/003_qubit-dynamics.ipynb

In [ ]:
def ramsey():
    

NameError: name 'qt' is not defined

In [ ]:
# coefficients



delta = 2 * np.pi
g = 0.25

# hamiltonian
H = delta / 2.0 * qt.sigmax()

# list of collapse operators
c_ops = [np.sqrt(g) * qt.sigmaz()]

# initial state
psi0 = qt.basis(2, 0)

# times
tlist = np.linspace(0, 5, 100)

NameError: name 'np' is not defined

In [2]:
res = mesolve(H, psi0, tlist, c_ops, [sigmaz()])


NameError: name 'mesolve' is not defined